# Sample code for multi-agent RAG with langGraph

In [ ]:
from typing import TypedDict, Optional

class GraphState(TypedDict):
    total_iterations: Optional[int] = None
    input_feedback: Optional[str] = None  # Feedback from InputValidator
    location_feedback: Optional[str] = None  # Feedback from LocationSelector
    budget_feedback: Optional[str] = None  # Feedback from BudgetReviewer
    schedule_feedback: Optional[str] = None  # Feedback from Scheduler
    
    # Additional fields for the user inputs
    start_time: Optional[str] = None  # Start time
    end_time: Optional[str] = None  # End time
    indoor_outdoor: Optional[str] = None  # Indoor or outdoor preference
    country: Optional[int] = None  # Country
    budget: Optional[float] = None  # User's budget
    food_preference: Optional[str] = None  # Food preferences (e.g., vegetarian, etc.)
    activity_preference: Optional[str] = None  # Activity preference (e.g., relaxing, adventurous)


In [ ]:
# Common LLM agent
import requests
from dotenv import load_dotenv
import os


class LLM:
    def __init__(self):
        load_dotenv()
        self.model_url = 'https://api.openai.com/v1/chat/completions'
        self.api_key = os.getenv('API_KEY')
        self.headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.api_key}'
        }

    def get_llm_response(self, prompt):
        data = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }

        llm_response = ""
        response = requests.post(self.model_url, headers=self.headers, json=data)
        if response.status_code == 200:
            llm_response = response.json()['choices'][0]['message']['content']
        else:
            print("Error:", response.status_code, response.text)

        return llm_response


In [ ]:
# Input validator agent
class InputValidator:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.reviewer_prompt = (
            "Please validate the following user inputs.\n"
            "Return a structured list of valid or invalid inputs based on the user response. "
            "Consider the following factors for location suggestions:\n"
            "1. Type of date (casual, formal, adventurous, etc.)\n"
            "2. Indoor or outdoor preference\n"
            "3. Country for the date (e.g., France, Singapore, etc.)\n"
            "4. Food preferences or dietary restrictions (if any)\n"
            "5. Preferred activity type (relaxing, fun, adventurous, etc.)\n"
            "6. Weather conditions (indoor vs outdoor based on forecast)\n"
            "Inputs: {}"
        )

    def validate_input(self, state):
        # Collect the necessary inputs from the state
        original_query = state.get('original_query', '').strip()
        date_type = state.get('date_type', 'casual')  # Type of date (casual, formal, etc.)
        indoor_outdoor = state.get('indoor_outdoor', 'both')  # Indoor or outdoor preference
        country = state.get('country', 'Singapore')  # Max travel distance in km
        budget = state.get('budget', 200.0)  # User's budget range
        food_preferences = state.get('food_preferences', '')  # Food preferences (e.g., vegetarian, etc.)
        activity_type = state.get('activity_type', 'relaxing')  # Activity preference (fun, adventurous, etc.)

        # Prepare the input prompt with collected user preferences
        user_input = (
            f"Date Type: {date_type}, "
            f"Indoor/Outdoor Preference: {indoor_outdoor}, "
            f"Country of activities: {country}, "
            f"Budget: {budget}, "
            f"Food Preferences: {food_preferences}, "
            f"Activity Type: {activity_type}, "
        )

        # Get the agent's feedback using the updated prompt
        agent_feedback = self.llm_caller.get_llm_response(self.reviewer_prompt.format(user_input))

        # Add the feedback and return the updated state
        return {
            'original_query': original_query,
            'updated_query': state.get('updated_query', ''),
            'input_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0),
            'date_type': date_type,
            'indoor_outdoor': indoor_outdoor,
            'country': country,
            'budget': budget,
            'food_preferences': food_preferences,
            'activity_type': activity_type,
        }

# Scheduling agent
class SchedulingAgent:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.scheduling_prompt = (
            "Create a schedule for a date considering the following details: "
            "Country: {country}, Start Time: {start_time}, End Time: {end_time}, "
            "Activity Preferences: {activity_preference}. Ensure the plan is feasible and aligns with the provided inputs."
            "Input the specific locations if available from location feedback: {location_feedback} " 
            "If not, just general type of activities E.g. Dinner, Lunch, Supper, outdoor activity, indoor activity, etc. ."
            "You may take budget feedback into considerations if available: {budget_feedback}"
        )

    def schedule_date(self, state: GraphState) -> str:
        """
        Generate or adjust a schedule based on the current state.

        Args:
            state (GraphState): The current state of the planning process.

        Returns:
            str: Feedback or proposed schedule from the agent.
        """
        # Extract relevant details from the state
        country = state.get("country", "Singapore")
        start_time = state.get("start_time", "N/A")
        end_time = state.get("end_time", "N/A")
        activity_preference = state.get("activity_preference", "any")
        location_feedback = state.get("location_feedback", "No specific location yet").strip()
        budget_feedback = state.get("budget_feedback", "No budget feedback yet").strip()
        
        # Format the prompt for the LLM
        formatted_prompt = self.scheduling_prompt.format(
            country=country,
            start_time=start_time,
            end_time=end_time,
            activity_preference=activity_preference,
            location_feedback=location_feedback,
            budget_feedback=budget_feedback,
        )

        # Call the LLM for feedback or a schedule proposal
        agent_feedback = self.llm_caller.get_llm_response(formatted_prompt)

        # Add feedback to the state and return updated state
        return {
            'original_query': state.get('original_query', ''),
            'schedule_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0)
        }

# Location selection agent
class LocationSelector:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.location_prompt = (
            "Given the schedule feedback, if available from: {schedule_feedback}.\n"
            "And given the budget feedback, if available from: {budget_feedback}.\n"
            "Select suitable locations to fit into the schedule based on the user's preferences.\n"
            "Provide a brief location feedback for each location chosen. "
            "The user's preferences are: {user_input}.\n"
            
        )
    def select_location(self, state):
        # Retrieve the necessary values from the state
        indoor_outdoor = state.get('indoor_outdoor', '')
        food_preference = state.get('food_preference', '')
        activity_preference = state.get('activity_preference', '')
        

        # Prepare the user input for location selection based on their preferences
        user_input = (
            f"Indoor/Outdoor Preference: {indoor_outdoor}, "
            f"Food Preferences: {food_preference}, "
            f"Activity Preferences: {activity_preference}"
        )
        schedule_feedback = state.get('schedule_feedback', '').strip()
        budget_feedback = state.get('budget_feedback', '').strip()
        # Format the prompt for the LLM
        formatted_prompt = self.scheduling_prompt.format(
            user_input=user_input,
            schedule_feedback=schedule_feedback,
            budget_feedback=budget_feedback
        )

        agent_feedback = self.llm_caller.get_llm_response(self.location_prompt.format(formatted_prompt))

        # Add feedback to the state and return updated state
        return {
            'original_query': state.get('original_query', ''),
            'location_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0)
        }

        
# Budget review agent
class BudgetAgent:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.budget_prompt = (
            "Evaluate if the proposed schedule and locations fit within the user's budget.\n"
            "Budget: {budget}.\n"
            "Schedule Feedback if available: {schedule_feedback}.\n"
            "Location Feedback if available: {location_feedback}.\n"
            "Consider additional costs such as transportation, meals, and activity fees.\n"
            "Return your analysis and recommendations:\n"
            "1. Whether the budget is sufficient.\n"
            "2. Suggestions for adjustments if needed.\n"
        )

    def review_budget(self, state: GraphState) -> str:
        """
        Evaluate the current state against the user's budget and provide feedback.

        Args:
            state (GraphState): The current planning state.

        Returns:
            str: Feedback on the budget and recommendations.
        """
        # Extract relevant details from the state
        budget = state.get("budget", 200.0)  # User's specified budget
        schedule_feedback = state.get("schedule_feedback", "No schedule provided.").strip()
        location_feedback = state.get("location_feedback", "No locations provided.").strip()

        # Format the prompt for the LLM
        formatted_prompt = self.budget_prompt.format(
            budget=budget,
            schedule_feedback=schedule_feedback,
            location_feedback=location_feedback,
        )

        # Call the LLM for budget analysis
        agent_feedback = self.llm_caller.get_llm_response(formatted_prompt)

        # Add feedback to the state and return updated state
        return {
            'original_query': state.get('original_query', ''),
            'budget_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0)
        }


        



In [ ]:
class Evaluator:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.evaluator_prompt = (
            "You are an evaluator tasked with assessing the feasibility of a date plan based on the following constraints:\n"
            "1. Budget feedback: {budget_feedback}\n"
            "2. User exact budget: {budget}\n"
            "3. Location Feedback: {location_feedback}\n"
            "4. Schedule Feedback: {schedule_feedback}\n\n"
            "Please answer the following:\n"
            "a) Does the budget align with the proposed locations and activities?\n"
            "b) Are the selected locations feasible based on the schedule?\n"
            "c) Is the overall plan aligned with the user's preferences and constraints?\n\n"
            "Only output 'Yes' if all conditions are met and the plan is feasible; otherwise, output 'No'."
        )

    def evaluate_plan(self, state):
        """
        Evaluate the feasibility of the current date plan.

        Args:
            state (GraphState): The current state containing feedback from agents.

        Returns:
            str: The name of the next agent to proceed with ('scheduling_agent' or 'input_validator').
        """
        # Retrieve feedback from the state
        budget_feedback = state.get('budget_feedback', '').strip()
        location_feedback = state.get('location_feedback', '').strip()
        schedule_feedback = state.get('schedule_feedback', '').strip()
        # Retrieve user constraints for detailed evaluation
        budget = state.get('budget', 0)
        

        # Prepare the evaluation prompt
        formatted_prompt = self.evaluator_prompt.format(budget, budget_feedback, location_feedback, schedule_feedback)

        # Call the LLM to assess the feasibility of the plan
        evaluator_response = self.llm_caller.get_llm_response(formatted_prompt)

        # Determine next step based on evaluator response and constraints
        if evaluator_response.lower() == 'yes':
            return 'finalize_plan'  # All constraints satisfied; ready to finalize
        elif state.get('total_iterations', 0) > 4:
            return 'scheduling_agent'  # Allow scheduling if iterations exceed threshold
        else:
            return 'input_validator'  # Revisit input validation for adjustments



In [ ]:
from langgraph.graph import StateGraph, END

# Instantiate the workflow and LLM
dating_review_workflow = StateGraph(GraphState)
llm_caller = LLM()

# Define agents
input_validator = InputValidator(llm_caller=llm_caller)
location_selector = LocationSelector(llm_caller=llm_caller)
scheduling_agent = SchedulingAgent(llm_caller=llm_caller)
budget_reviewer = BudgetAgent(llm_caller=llm_caller)
evaluator = Evaluator(llm_caller=llm_caller)

# Add nodes for agents
dating_review_workflow.add_node("input_validator", agent=input_validator)
dating_review_workflow.add_node("location_selector", agent=location_selector)
dating_review_workflow.add_node("scheduling_agent", agent=scheduling_agent)
dating_review_workflow.add_node("budget_reviewer", agent=budget_reviewer)
dating_review_workflow.add_node("evaluator", agent=evaluator)

# Set entry point
dating_review_workflow.set_entry_point("input_validator")

# Add edges for transitions, including evaluator
dating_review_workflow.add_edge("input_validator", "location_selector")
dating_review_workflow.add_edge("location_selector", "scheduling_agent")
dating_review_workflow.add_edge("scheduling_agent", "budget_reviewer")
dating_review_workflow.add_edge("budget_reviewer", "evaluator")

# Add evaluation-based transitions
# If evaluation passes, end the workflow; otherwise, loop back as needed
dating_review_workflow.add_edge("evaluator", "scheduling_agent", condition=lambda state: state.get('next_agent') == 'scheduling_agent')
dating_review_workflow.add_edge("evaluator", "input_validator", condition=lambda state: state.get('next_agent') == 'input_validator')
dating_review_workflow.add_edge("evaluator", END, condition=lambda state: state.get('next_agent') == 'finalize_plan')

# Add END condition for budget reviewer (final step before evaluation)
dating_review_workflow.add_edge("budget_reviewer", "evaluator")
